In [1]:
import pyspark 
from pyspark import SparkContext
sc = SparkContext()
spark = pyspark.sql.SparkSession(sc, jsparkSession=None)

## Import File

In [2]:
df = spark.read.option('InferSchema', 'true').option('header', 'true').csv('BoardGameGeek_12-11-23.csv')

In [3]:
df.show()

+------+--------------------+----+----+-------+-------------+-----------+--------------------+--------------------+
|    ID|                Name|Year|Rank|Average|Bayes average|Users rated|                 URL|           Thumbnail|
+------+--------------------+----+----+-------+-------------+-----------+--------------------+--------------------+
|224517|   Brass: Birmingham|2018|   1|   8.61|        8.421|    42051.0|/boardgame/224517...|https://cf.geekdo...|
|161936|Pandemic Legacy: ...|2015|   2|   8.53|        8.386|    51975.0|/boardgame/161936...|https://cf.geekdo...|
|174430|          Gloomhaven|2017|   3|   8.61|        8.373|    60305.0|/boardgame/174430...|https://cf.geekdo...|
|342942|            Ark Nova|2021|   4|   8.53|         8.32|    37121.0|/boardgame/342942...|https://cf.geekdo...|
|233078|Twilight Imperium...|2017|   5|   8.61|        8.242|    22140.0|/boardgame/233078...|https://cf.geekdo...|
|167791|   Terraforming Mars|2016|   6|   8.37|        8.221|    94140.0

In [4]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Rank: integer (nullable = true)
 |-- Average: double (nullable = true)
 |-- Bayes average: double (nullable = true)
 |-- Users rated: double (nullable = true)
 |-- URL: string (nullable = true)
 |-- Thumbnail: string (nullable = true)



## Initialize SparkSQL

In [5]:
df.createOrReplaceTempView("boardgames")

In [6]:
spark.sql(
    '''
    select 
    id,
    name, 
    cast(year as int) year,
    rank, 
    average, 
    `boardgames`.`Bayes average`,
    cast(`boardgames`.`Users rated` as int) `Users rated`,
    "https://boardgamegeek.com" || url as url
    from boardgames
    ''').createOrReplaceTempView("boardgames")

In [7]:
spark.sql('''select * from boardgames''').show()

+------+--------------------+----+----+-------+-------------+-----------+--------------------+
|    id|                name|year|rank|average|Bayes average|Users rated|                 url|
+------+--------------------+----+----+-------+-------------+-----------+--------------------+
|224517|   Brass: Birmingham|2018|   1|   8.61|        8.421|      42051|https://boardgame...|
|161936|Pandemic Legacy: ...|2015|   2|   8.53|        8.386|      51975|https://boardgame...|
|174430|          Gloomhaven|2017|   3|   8.61|        8.373|      60305|https://boardgame...|
|342942|            Ark Nova|2021|   4|   8.53|         8.32|      37121|https://boardgame...|
|233078|Twilight Imperium...|2017|   5|   8.61|        8.242|      22140|https://boardgame...|
|167791|   Terraforming Mars|2016|   6|   8.37|        8.221|      94140|https://boardgame...|
|316554|      Dune: Imperium|2020|   7|   8.42|        8.209|      39698|https://boardgame...|
|291457|Gloomhaven: Jaws ...|2020|   8|   8.47|   

## Highest Ranked Board Game by Year

In [8]:
highestrankedgamebyyear = spark.sql(
    '''
    select * 
    from ( 
        select  row_number() over(partition by year order by rank) as row_num,
                boardgames.*
        from boardgames
        ) bg
    where row_num = 1 
    and year <= year(current_date) /*some games in dataset were created in 3000 BC, need to remove these dates*/
    order by year desc
    ''')

In [9]:
highestrankedgamebyyear.show()

+-------+------+--------------------+----+----+-------+-------------+-----------+--------------------+
|row_num|    id|                name|year|rank|average|Bayes average|Users rated|                 url|
+-------+------+--------------------+----+----+-------+-------------+-----------+--------------------+
|      1|321608|Hegemony: Lead Yo...|2023| 179|   8.54|        7.357|       3623|https://boardgame...|
|      1|295770|          Frosthaven|2022|  42|   8.87|        7.788|       5680|https://boardgame...|
|      1|342942|            Ark Nova|2021|   4|   8.53|         8.32|      37121|https://boardgame...|
|      1|316554|      Dune: Imperium|2020|   7|   8.42|        8.209|      39698|https://boardgame...|
|      1|266507|Clank! Legacy: Ac...|2019|  22|   8.56|        7.948|       8720|https://boardgame...|
|      1|224517|   Brass: Birmingham|2018|   1|   8.61|        8.421|      42051|https://boardgame...|
|      1|174430|          Gloomhaven|2017|   3|   8.61|        8.373|    

In [10]:
highestrankedgamebyyear.write.option('header','true').csv('highestrankedgamebyyear.csv', mode='overwrite')

## Highest Rated Board Game by Year

In [11]:
bestratedgamebyyear = spark.sql(
    '''
    select * 
    from ( 
        select  row_number() over(partition by year order by rank) as row_num,
                boardgames.*
        from boardgames
        ) bg
    where row_num = 1 
    and year <= year(current_date) /*some games in dataset were created in 3000 BC, need to remove these dates*/
    order by year desc
    ''')

In [12]:
bestratedgamebyyear.show()

+-------+------+--------------------+----+----+-------+-------------+-----------+--------------------+
|row_num|    id|                name|year|rank|average|Bayes average|Users rated|                 url|
+-------+------+--------------------+----+----+-------+-------------+-----------+--------------------+
|      1|321608|Hegemony: Lead Yo...|2023| 179|   8.54|        7.357|       3623|https://boardgame...|
|      1|295770|          Frosthaven|2022|  42|   8.87|        7.788|       5680|https://boardgame...|
|      1|342942|            Ark Nova|2021|   4|   8.53|         8.32|      37121|https://boardgame...|
|      1|316554|      Dune: Imperium|2020|   7|   8.42|        8.209|      39698|https://boardgame...|
|      1|266507|Clank! Legacy: Ac...|2019|  22|   8.56|        7.948|       8720|https://boardgame...|
|      1|224517|   Brass: Birmingham|2018|   1|   8.61|        8.421|      42051|https://boardgame...|
|      1|174430|          Gloomhaven|2017|   3|   8.61|        8.373|    

In [13]:
bestratedgamebyyear.write.option('header','true').csv('bestratedgamebyyear.csv', mode='overwrite')

## Most Rated Board Game by Year

In [14]:
mostratedgamebyyear = spark.sql(
    '''
    select * 
    from ( 
        select  row_number() over(partition by year order by rank) as row_num,
                boardgames.*
        from boardgames
        ) bg
    where row_num = 1 
    and year <= year(current_date) /*some games in dataset were created in 3000 BC, need to remove these dates*/
    order by year desc
    ''')

In [15]:
mostratedgamebyyear.show()

+-------+------+--------------------+----+----+-------+-------------+-----------+--------------------+
|row_num|    id|                name|year|rank|average|Bayes average|Users rated|                 url|
+-------+------+--------------------+----+----+-------+-------------+-----------+--------------------+
|      1|321608|Hegemony: Lead Yo...|2023| 179|   8.54|        7.357|       3623|https://boardgame...|
|      1|295770|          Frosthaven|2022|  42|   8.87|        7.788|       5680|https://boardgame...|
|      1|342942|            Ark Nova|2021|   4|   8.53|         8.32|      37121|https://boardgame...|
|      1|316554|      Dune: Imperium|2020|   7|   8.42|        8.209|      39698|https://boardgame...|
|      1|266507|Clank! Legacy: Ac...|2019|  22|   8.56|        7.948|       8720|https://boardgame...|
|      1|224517|   Brass: Birmingham|2018|   1|   8.61|        8.421|      42051|https://boardgame...|
|      1|174430|          Gloomhaven|2017|   3|   8.61|        8.373|    

In [16]:
mostratedgamebyyear.write.option('header','true').csv('mostratedgamebyyear.csv', mode='overwrite')